aaa

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import (CountVectorizer, TfidfVectorizer)
from sklearn.naive_bayes import (GaussianNB, MultinomialNB)
import re 
import nltk 
from sklearn.metrics import (accuracy_score, ConfusionMatrixDisplay)
from nltk.corpus import stopwords
from nltk import ngrams
nltk.download("stopwords")
stop_words = stopwords.words("spanish") # stopwords palabras de relleno

# Análisis de texto clásico ciertas palabras no añaden contexto y en base al resto de palabras podemos entender las relaciones que permitan hacer la clasficación
# Función para quitar los stopwords del data frame
def remove_stopwords(text):
    clean_text = []
    text = text.lower()
    if text is not None:
        words = text.split()
        for word in words:
            if word not in stop_words:
                clean_text.append(word)
        return " ".join(clean_text)
    else:
        return None


def remove_symbols(text):
    if text is not None:
        text = re.sub(r'''[.,?*&'"+${}-]''','',text) # Sub sustituir 
        return text
    else:
        return None

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/codespace/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [2]:
# Open the Dataset
df_raw = pd.read_csv("archivo_combinado.csv")
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 702 entries, 0 to 701
Data columns (total 21 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Identificador de la publicación  702 non-null    int64  
 1   Identificador de la cuenta       702 non-null    int64  
 2   Nombre de usuario de la cuenta   702 non-null    object 
 3   Nombre de la cuenta              702 non-null    object 
 4   Descripción                      302 non-null    object 
 5   Duración (segundos)              702 non-null    int64  
 6   Hora de publicación              702 non-null    object 
 7   Enlace permanente                13 non-null     object 
 8   Tipo de publicación              702 non-null    object 
 9   Comentario sobre los datos       0 non-null      float64
 10  Fecha                            702 non-null    object 
 11  Alcance                          642 non-null    float64
 12  Me gusta              

In [3]:
# Create a Dataframe only with the id and description
selected_columns = ["Identificador de la publicación", "Descripción"]
df_interim = df_raw[selected_columns]
df_interim.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 702 entries, 0 to 701
Data columns (total 2 columns):
 #   Column                           Non-Null Count  Dtype 
---  ------                           --------------  ----- 
 0   Identificador de la publicación  702 non-null    int64 
 1   Descripción                      302 non-null    object
dtypes: int64(1), object(1)
memory usage: 11.1+ KB


In [6]:
# Remove rows with NaN
df_interim = df_interim.dropna(subset=["Descripción"])
df_interim.info()


<class 'pandas.core.frame.DataFrame'>
Index: 302 entries, 0 to 701
Data columns (total 2 columns):
 #   Column                           Non-Null Count  Dtype 
---  ------                           --------------  ----- 
 0   Identificador de la publicación  302 non-null    int64 
 1   Descripción                      302 non-null    object
dtypes: int64(1), object(1)
memory usage: 7.1+ KB


In [9]:
# Clean data
df_interim["message_clean"] = df_interim["Descripción"].apply(lambda x: remove_stopwords(x))
df_interim["message_clean"] = df_interim["message_clean"].apply(lambda x: remove_symbols(x))
df_interim["message_clean"] = df_interim["message_clean"].str.replace(":", "")
df_interim["message_clean"] = df_interim["message_clean"].str.replace("!", "")
df_interim["message_clean"] = df_interim["message_clean"].str.replace("¡", "")
df_interim["message_clean"] = df_interim["message_clean"].str.replace("(", "")
df_interim["message_clean"] = df_interim["message_clean"].str.replace(")", "")
df_interim.info()

<class 'pandas.core.frame.DataFrame'>
Index: 302 entries, 0 to 701
Data columns (total 3 columns):
 #   Column                           Non-Null Count  Dtype 
---  ------                           --------------  ----- 
 0   Identificador de la publicación  302 non-null    int64 
 1   Descripción                      302 non-null    object
 2   message_clean                    302 non-null    object
dtypes: int64(1), object(2)
memory usage: 9.4+ KB


In [10]:
df_interim.sample(10)

,Identificador de la publicación,Descripción,message_clean
413,18333787006124814,Todos los sueños tienen significado \nMis sueños:,sueños significado sueños
575,18469601890008634,Hoy agradezco la oportunidad que tuve de asist...,hoy agradezco oportunidad asistir vidcon apren...
444,17936136686789609,El café te da energía; tus sueños te dan direc...,café da energía; sueños dan dirección despiert...
454,18313632556093128,¡Hoy voy a probar \nalgo nuevo! ¡Te cuento \np...,hoy voy probar nuevo te cuento pronto cómo fue...
401,18265333957300216,Cafecito en\n#houseofinstagram,cafecito #houseofinstagram
559,18089224099490194,A punto de comenzar entrenamiento presencial p...,punto comenzar entrenamiento presencial @ekrme...
464,17979505067082348,Cuando la vida es un sueño,vida sueño
90,18272236114082552,Está bien bonito 😻,bien bonito 😻
348,18059875228340678,¡Ya casi empieza!,ya casi empieza
604,17893072272017102,¡Mañana voy\nA hacer un live!,mañana voy hacer live


In [11]:
import spacy

# Carga el modelo preentrenado en español
nlp = spacy.load("es_core_news_sm")

# Función para obtener el sentimiento de un texto
def obtener_sentimiento(texto):
    doc = nlp(texto)
    return doc.sentiment

# Supongamos que df_interim es tu DataFrame con la columna "message_clean"
df_interim['sentimiento'] = df_interim['message_clean'].apply(lambda x: obtener_sentimiento(x) if isinstance(x, str) else None)

# Mostrar el DataFrame con la nueva columna de sentimiento
print(df_interim.head())


    Identificador de la publicación  \
0                 18003913832026677   
2                 17943046658757431   
4                 18112602532344424   
12                18077532931430567   
13                18092816236380356   

                                          Descripción  \
0                ¡Feliz día\nInternacional del\nCafé!   
2              También tu haces mi vida muy feliz 😀❤️   
4   ¿Quieres recibir un ritual para incubar un sue...   
12                          Está más peinada que yo 😂   
13      Al curso intensivo "Soñando el futuro juntos"   

                                      message_clean  sentimiento  
0                      feliz día internacional café          0.0  
2                              haces vida feliz 😀❤️          0.0  
4   ¿quieres recibir ritual incubar sueño guíe 2024          0.0  
12                                        peinada 😂          0.0  
13            curso intensivo soñando futuro juntos          0.0  
